In [ ]:
import os
import numpy as np
from PIL import Image
from skimage.transform import resize, rotate
from scipy.ndimage import gaussian_filter
import multiprocessing as mp

def process_image(filename):
    original_image = np.array(Image.open(filename))
    sequences = []
    target_shape = (64, 64, 1)
    n, m = 3, 20

    for i in range(n):
        print(i,n)
        frames = []
        time_wave_start = 2 * (np.random.rand() - 0.5)
        time_wave_end = (2 + time_wave_start) + 2 * (np.random.rand() - 0.5)
        time_wave = np.sin(np.linspace(time_wave_start * np.pi, time_wave_end * np.pi, m))
        max_elongation = np.random.rand() * 0.3
        width_weight = np.random.rand() * 0.3
        blur_amount = np.random.rand() * 50
        flip_or_not = np.round(np.random.rand())
        rot_angle = (np.random.rand() - 0.5) * 10
        black_frame_size = (int(original_image.shape[0] * (1 + max_elongation)), int(original_image.shape[1] * (1 + max_elongation)), original_image.shape[2])

        for j in range(m):
            elongation_factor_height = 1.0 + max_elongation * time_wave[j]
            elongation_factor_width = 1.0 + max_elongation * width_weight * time_wave[j]
            resized_image = resize(original_image, (int(original_image.shape[0] * elongation_factor_height), int(original_image.shape[1] * elongation_factor_width), original_image.shape[2]), anti_aliasing=True, preserve_range=True, mode='symmetric')
            sigma = blur_amount * (1 - np.abs(time_wave[j]))
            output = gaussian_filter(resized_image, sigma=(sigma, sigma * width_weight, 0))
            resized_height, resized_width = output.shape[:2]
            black_image = np.zeros(black_frame_size)
            vertical_start = black_frame_size[0] - resized_height
            horizontal_start = (black_frame_size[1] - resized_width) // 2
            black_image[vertical_start:vertical_start + resized_height, horizontal_start:horizontal_start + resized_width, :] = output
            frame = np.mean(black_image[:, :, :3], axis=2, keepdims=True)
            if flip_or_not == 1:
                frame = np.fliplr(frame)
            frame = rotate(frame, rot_angle, preserve_range=True, mode='edge')
            frame = resize(frame, target_shape, anti_aliasing=True)
            frames.append(frame)

        sequences.append(frames)
    return sequences

def main():
    directory_path = "../segmentation_extraction/extracted_slices"
    pool = mp.Pool(mp.cpu_count())
    all_filenames = [os.path.join(directory_path, f) for f in os.listdir(directory_path) if f.endswith('.png')]
    all_sequences = pool.map(process_image, all_filenames)
    pool.close()
    pool.join()
    np.save('output_sequences.npy', np.array(all_sequences))
    print("Sequences saved successfully.")

if __name__ == '__main__':
    main()
